<a href="https://colab.research.google.com/github/aidan-lew/junk-drawer/blob/main/finrl_fingpt_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These are the recourses to make a venv for colab:
https://stackoverflow.com/questions/74244113/how-to-make-virtual-environment-on-google-colab-that-can-be-saved-on-drive
https://colab.research.google.com/drive/1Vg005uKhgt-ZwyQ11-BGceqY0q07qIzF

In [1]:
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install torch
!pip install googlesearch-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4

# Load FinGPT

In [2]:
from huggingface_hub import notebook_login

In [3]:
notebook_login()

In [4]:
#have to have access to LLama -  via hugging face
### THIS IS ABlE TO RUN ON a T4!!!
# https://github.com/AI4Finance-Foundation/FinGPT/tree/master/fingpt/FinGPT_Forecaster
# https://github.com/AI4Finance-Foundation/FinGPT/blob/master/FinGPT_Inference_Llama2_13B_falcon_7B_for_Beginners.ipynb
### see how the flow works for these, look at the hugging face page dataset for training.

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,   # optional if you have enough VRAM
)
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')

gpt_model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora')
gpt_model = gpt_model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/40.0M [00:00<?, ?B/s]

NameError: name 'model' is not defined

# Whole pipeline

## Scraper

In [44]:
from googlesearch import search
from bs4 import BeautifulSoup
import requests

def scrape_news_headlines(ticker, num_articles):
  query = f"{ticker} news" #####find a better url/website because the current one just gets the google web results
  search_results = search(query)
  articles = []
  count = 0
  for url in search_results:
      if count >= num_articles:
          break
      try:
          response = requests.get(url, verify=False, headers={'User-Agent':'Mozilla/5.0'}, #timeout=10
                                  )
          soup = BeautifulSoup(response.text, 'html.parser')
          title = soup.find('title').text if soup.find('title') else 'No title'
          articles.append({'title': title, 'link': url})
          count += 1
      except Exception as e:
          print(f"Error processing {url}: {e}")
  return articles


## Analyzer

In [63]:
def analyze_news_sentiment(articles):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  gpt_model.to(device)  # Move the model to the device

  sentiments = []
  for article in articles:
      inputs = tokenizer(article['title'], return_tensors='pt', max_length=512, truncation=True)
      inputs = {k: v.to(device) for k, v in inputs.items()}  # Move the input tensors to the device
      outputs = gpt_model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
      sentiment = torch.nn.functional.softmax(outputs.logits, dim=1)
      sentiments.append(sentiment.detach().cpu().numpy())  # Move the sentiment tensor back to the CPU before converting to numpy
      #print(sentiments)
  return sentiments

## Predictor

In [59]:
import numpy as np
import yfinance as yf
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd

def make_stock_price_prediction(ticker, sentiments):
    stock_data = yf.download(ticker, period='1d', interval='1m')

    # Extract the positive sentiment score (index 1) from each softmax output
    sentiment_scores = [sentiment[0, 1] for sentiment in sentiments]

    # Create a 1D numpy array from the sentiment scores
    sentiments = np.array(sentiment_scores)

    sentiment_mean = np.mean(sentiments)
    #print(f"Sentiment mean: {sentiment_mean}")

    stock_data['sentiment'] = [sentiment_mean] * len(stock_data)  # Create a new column with the same value repeated for each row
    X = stock_data[['sentiment']]
    y = stock_data['Close']

    #print(f"X shape: {X.shape}")
    #print(f"y shape: {y.shape}")

    # Fit a HistGradientBoostingRegressor model
    fin_model = HistGradientBoostingRegressor()
    fin_model.fit(X, y)

    # Make a prediction
    prediction_input = pd.DataFrame({'sentiment': [sentiment_mean]})
    #print(f"Prediction input shape: {prediction_input.shape}")

    prediction = fin_model.predict(prediction_input)

    #print(f"Prediction: {prediction}")

    return prediction[0]  # Return the scalar prediction


## Pipeline Function

In [83]:
def prediction_pipeline():
  ticker = input('ticker')
  num_articles = int(input('number of articles'))
  articles = scrape_news_headlines(ticker, num_articles)
  sentiments = analyze_news_sentiment(articles)
  prediction = make_stock_price_prediction(ticker, sentiments)
  print('\n' f"Predicted stock price for {ticker}: {prediction}")
  #return ticker
  #return num_articles
  #return prediction

In [84]:
prediction_pipeline()

tickerMSFT
number of articles10


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110


Predicted stock price for MSFT: 448.7495869140625


# Piece by piece

## Scraper

In [6]:
####### News sources - what about econ data, general news, etc?
#### googlenews API (https://pypi.org/project/GoogleNews/)
### Rueters (https://www.reuters.com/business/)
### https://www.bloomberg.com/
from googlesearch import search
from bs4 import BeautifulSoup
import requests

def scrape_news_headlines(ticker, num_articles):
  query = f"{ticker} news" #####find a better url/website because the current one just gets the google web results
  search_results = search(query)
  articles = []
  count = 0
  for url in search_results:
      if count >= num_articles:
          break
      try:
          response = requests.get(url, verify=False, headers={'User-Agent':'Mozilla/5.0'}, #timeout=10
                                  )
          soup = BeautifulSoup(response.text, 'html.parser')
          title = soup.find('title').text if soup.find('title') else 'No title'
          articles.append({'title': title, 'link': url})
          count += 1
      except Exception as e:
          print(f"Error processing {url}: {e}")
  return articles

In [85]:
ticker = 'AAPL'
num_articles = 10

articles = scrape_news_headlines(ticker, num_articles)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110

In [8]:
articles

[{'title': 'AAPL: Apple Inc - Stock Price, Quote and News - CNBC',
  'link': 'https://www.cnbc.com/quotes/AAPL'},
 {'title': 'Apple (AAPL) News Headlines | Nasdaq',
  'link': 'https://www.nasdaq.com/market-activity/stocks/aapl/news-headlines'},
 {'title': 'Apple Inc. (AAPL) Latest Stock News & Headlines - Yahoo Finance',
  'link': 'https://finance.yahoo.com/quote/AAPL/news/'},
 {'title': 'Attention Required! | Cloudflare',
  'link': 'https://investor.apple.com/investor-relations/default.aspx'},
 {'title': 'marketwatch.com',
  'link': 'https://www.marketwatch.com/investing/stock/aapl'},
 {'title': 'Apple (AAPL) Stock Price, Quote, News & History | Nasdaq',
  'link': 'https://www.nasdaq.com/market-activity/stocks/aapl'},
 {'title': 'Bloomberg - Are you a robot?',
  'link': 'https://www.bloomberg.com/quote/AAPL:US'},
 {'title': 'Apple Inc. (AAPL) Stock Price, Quote, News & Analysis',
  'link': 'https://seekingalpha.com/symbol/AAPL'},
 {'title': 'Apple Inc. (AAPL) Stock Price, News, Quote 

## Analyzer

In [52]:
def analyze_news_sentiment(articles):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  gpt_model.to(device)  # Move the model to the device

  sentiments = []
  for article in articles:
      inputs = tokenizer(article['title'], return_tensors='pt', max_length=512, truncation=True)
      inputs = {k: v.to(device) for k, v in inputs.items()}  # Move the input tensors to the device
      outputs = gpt_model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
      sentiment = torch.nn.functional.softmax(outputs.logits, dim=1)
      sentiments.append(sentiment.detach().cpu().numpy())  # Move the sentiment tensor back to the CPU before converting to numpy
      print(sentiments)
  return sentiments

In [27]:
sentiments = analyze_news_sentiment(articles)

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32)]
[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, .

In [28]:
sentiments

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
          1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
         [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
          1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
         [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
          2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
         ...,
         [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
          2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
         [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
          2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
         [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
          5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32),
 array([[[6.4145797e-04, 5.0152973e-03, 3.4871415e-04, ...,
          4.5422399e-03, 8.1572525e-02, 1.2902316e-03],
         [9.0460083e-04, 1.3417624e-04, 7.4513872e-07, ...,
          6.7129950e-03, 3.6121567e-03, 3.7188535e-03],
         [1.1063426e-02, 9.1692823e-04, 

## Predictor

In [53]:
import numpy as np
import yfinance as yf
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd

def make_stock_price_prediction(ticker, sentiments):
    stock_data = yf.download(ticker, period='1d', interval='1m')

    # Extract the positive sentiment score (index 1) from each softmax output
    sentiment_scores = [sentiment[0, 1] for sentiment in sentiments]

    # Create a 1D numpy array from the sentiment scores
    sentiments = np.array(sentiment_scores)

    sentiment_mean = np.mean(sentiments)
    print(f"Sentiment mean: {sentiment_mean}")

    stock_data['sentiment'] = [sentiment_mean] * len(stock_data)  # Create a new column with the same value repeated for each row
    X = stock_data[['sentiment']]
    y = stock_data['Close']

    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")

    # Fit a HistGradientBoostingRegressor model
    fin_model = HistGradientBoostingRegressor()
    fin_model.fit(X, y)

    # Make a prediction
    prediction_input = pd.DataFrame({'sentiment': [sentiment_mean]})
    print(f"Prediction input shape: {prediction_input.shape}")

    prediction = fin_model.predict(prediction_input)

    print(f"Prediction: {prediction}")

    return prediction[0]  # Return the scalar prediction


In [39]:
make_stock_price_prediction(ticker, sentiments)

[*********************100%%**********************]  1 of 1 completed

Sentiment mean: 0.105434849858284
X shape: (99, 1)
y shape: (99,)
Prediction input shape: (1, 1)
Prediction: [209.95151697]


209.9515169702395

## Pipeline Function

In [80]:
def prediction_pipeline():
  ticker = input('ticker')
  num_articles = int(input('number of articles'))
  articles = scrape_news_headlines(ticker, num_articles)
  sentiments = analyze_news_sentiment(articles)
  prediction = make_stock_price_prediction(ticker, sentiments)
  print('\n' f"Predicted stock price for {ticker}: {prediction}")
  return ticker
  return num_articles
  return prediction

In [81]:
prediction_pipeline()

tickerAAPL
number of articles10


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110


Predicted stock price for AAPL: 209.9384884717988


'AAPL'

In [55]:
ticker = 'AAPL'
num_articles = 10
prediction_pipeline(ticker, num_articles)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32)]
[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, .

[*********************100%%**********************]  1 of 1 completed

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32), array([[[1.56065449e-04, 2.30889302e-03, 7.26893340e-05, ...,
         1.96373928e-03, 9.80778132e-03, 2.92713230e-04],
        [2.20087561e-04, 6.15299141e-05, 1.56542086e-07, ...,
         2.90221884e-03, 4.34303569e-04, 8.43691756e-04],
        [3.48229078e-05, 5.06131255e-05, 1.0

209.94394170587714

In [40]:
ticker = 'AAPL'
num_articles = 10

articles = scrape_news_headlines(ticker, num_articles)
sentiments = analyze_news_sentiment(articles)
prediction = make_stock_price_prediction(ticker, sentiments)

print(f"Predicted stock price for {ticker}: {prediction}")

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32)]
[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, .

[*********************100%%**********************]  1 of 1 completed

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32), array([[[1.56065449e-04, 2.30889302e-03, 7.26893340e-05, ...,
         1.96373928e-03, 9.80778132e-03, 2.92713230e-04],
        [2.20087561e-04, 6.15299141e-05, 1.56542086e-07, ...,
         2.90221884e-03, 4.34303569e-04, 8.43691756e-04],
        [3.48229078e-05, 5.06131255e-05, 1.0

# With FinRL


# New Section